## Random forest regressor best params (this will continue 5 or 6 Random forest regression models from Initial modleing to see If I can get over .80 R2 score with random forest)

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

#### Read in

In [13]:
filepath = '../../processed_data/all_merged.csv'
all_enviro = pd.read_csv(filepath)

#### testing different GS parameters

In [14]:
X = all_enviro[['total population','ozone','pm2.5','diesel pm','pesticides','traffic','cleanup sites',
 'groundwater threats','haz. waste','imp. water bodies','solid waste','pollution burden',
'education','linguistic isolation','poverty','pop. char. ','drinking water',
 'tox. release','unemployment','ces_per','housing burden','est total',
 'est gen','est cold','est farm','est other']]
y = all_enviro['asthma']

###### clean before Splitting

In [15]:
X=pd.get_dummies(X)
X=X.fillna(X.median())

#get rid of missing target columns
X = X[~y.isna()]
y = y[~y.isna()]

In [16]:
X_train, X_test, y_train,y_test = train_test_split(X,y,random_state= 42)

###### Standard Scaler & fill with median

In [17]:
X_train=X_train.fillna(X_train.median())
X_test = X_test.fillna(X_test.median())

sc_X = StandardScaler()

X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [18]:
forest = RandomForestRegressor().fit(X_train,y_train)

In [19]:
print(f'Train R2 {forest.score(X_train,y_train)}')
print(f'Test R2 {forest.score(X_test,y_test)}')

Train R2 0.9629998032939701
Test R2 0.7288568738080135


#### Grid search different parameters

In [27]:
gsc = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid={ 
            "n_estimators"      : [10,20,30],
            "max_features"      : ["auto", "sqrt", "log2"],
            "max_depth" : [2,4],
            "bootstrap": [True, False],
            "min_samples_leaf": [3,5],
            },
            cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

In [28]:
forest_result = gsc.fit(X_train,y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


In [29]:
forest = RandomForestRegressor(**gsc.best_params_).fit(X_train, y_train)

In [32]:
best=gsc.best_params_
best

{'bootstrap': True,
 'max_depth': 4,
 'max_features': 'auto',
 'min_samples_leaf': 3,
 'n_estimators': 20}

In [31]:
print(f'Train accuracy {forest.score(X_train,y_train)}')
print(f'Test accuracy {forest.score(X_test,y_test)}')

Train accuracy 0.587234877563676
Test accuracy 0.5566669405395088


- My parameters were not helpful, too much added noise.

### Best features in data set